# Standard Model ( with oversampling ) 

In [32]:
import pandas as pd 
import numpy as np 

In [33]:
train=pd.read_csv('/content/drive/MyDrive/univ.ai/univ_data/Training Data.csv')
#test=pd.read_csv('')

In [34]:
def preprocessing(df):
  df=pd.get_dummies(data=df,columns=["house_ownership","married","car_ownership"],drop_first=True)
  df.drop(["profession","city","state"],axis=1,inplace=True)
  return df

In [35]:
train=preprocessing(train)

X = train.drop('risk_flag', axis=1)
y = train['risk_flag']

from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler(sampling_strategy='minority')
X_over, y_over = oversample.fit_resample(X,y)
X_over=pd.DataFrame(X_over,columns=X.columns)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [36]:
X_train, X_test, y_train, y_test = train_test_split(X_over, y_over, test_size=0.2,shuffle=True)

In [38]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score
classifiers = [
    KNeighborsClassifier(10),
    LogisticRegression(),
    RandomForestClassifier(),
    #GradientBoostingClassifier()
    ]
for classifier in classifiers:
    pipe = Pipeline(steps=[('classifier', classifier)])
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3,random_state=1)
    scores = cross_val_score(pipe, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)
    score = np.mean(scores)
    print(classifier)
    print('Roc Auc: %.3f' % score)

    pipe.fit(X_train,y_train)
    y_pred_test=pipe.predict(X_test)
    auc_test = roc_auc_score(y_test, y_pred_test)
    print('ROC AUC test: %f' % auc_test)
    print("==========================")

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')
Roc Auc: 0.792
ROC AUC test: 0.731935
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
Roc Auc: 0.522
ROC AUC test: 0.513448


/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
Roc Auc: 0.999
ROC AUC test: 0.981064


In [39]:
from sklearn.model_selection import GridSearchCV
grid_params={
    'n_neighbors':[3,5,11,15,21],
    'weights':['uniform','distance'],
    'metric':['euclidean','manhattan']
}
gs=GridSearchCV(KNeighborsClassifier(),grid_params,verbose=1,cv=3,n_jobs=-1)

gs_results=gs.fit(X_over,y_over)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  3.7min finished


In [45]:
test=pd.read_csv("/content/drive/MyDrive/univ.ai/univ_data/Test Data.csv")
ID=test["id"]
test=preprocessing(test)
predictions=gs_results.predict(test)

result=pd.DataFrame()
result["id"]=ID
result["risk_flag"]=predictions

In [48]:
result.to_csv("predictions2.csv",index=False)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_over,y_over)

In [ ]:
test=preprocessing(test)
predictions=knn.predict(test)

result=pd.DataFrame()
result["id"]=ID
result["risk_flag"]=predictions

# Model with Feature Enginnering 

In [24]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

In [25]:
train=pd.read_csv('/content/drive/MyDrive/univ.ai/univ_data/Training Data.csv')
#test=pd.read_csv('')

In [26]:
def preprocessing(df):
  df=pd.get_dummies(data=df,columns=["house_ownership","married","car_ownership"],drop_first=True)
  df.drop(["profession","city","state"],axis=1,inplace=True)
  return df

In [27]:
train=preprocessing(train)

X = train.drop('risk_flag', axis=1)
y = train['risk_flag']

from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler(sampling_strategy='minority')
X_over, y_over = oversample.fit_resample(X,y)
X_over=pd.DataFrame(X_over,columns=X.columns)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [28]:
X_train, X_test, y_train, y_test = train_test_split(X_over, y_over, test_size=0.2,shuffle=True)

In [29]:
transforms = list()
transforms.append(('mms', MinMaxScaler()))
# transforms.append(('ss', StandardScaler()))
# transforms.append(('rs', RobustScaler()))
transforms.append(('qt', QuantileTransformer(n_quantiles=100, output_distribution='normal')))
transforms.append(('kbd', KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')))
transforms.append(('pca', PCA(n_components=7)))
transforms.append(('svd', TruncatedSVD(n_components=7)))
# create the feature union
fu = FeatureUnion(transforms)

In [30]:
X_train_new=fu.fit_transform(X_train)
X_test_new=fu.transform(X_test)

In [31]:
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score
gbc=GradientBoostingClassifier()
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(gbc, X_train_new, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)
print("Roc Auc:",np.mean(scores))

Roc Auc: 0.7346388099008238


In [ ]:
gbc.fit(X_train_new,y_train)
y_pred_test=pipe.predict(X_test_new)
auc_test = roc_auc_score(y_test, y_pred_test)
print('ROC AUC test: %f' % auc_test)

In [ ]:
test=pd.read_csv("/content/drive/MyDrive/univ.ai/univ_data/Test Data.csv")
ID=test["id"]

test=preprocessing(test)
test=fu.transform(test)

# Undersampling

In [1]:
import pandas as pd 
import numpy as np 

In [3]:
train=pd.read_csv('/content/drive/MyDrive/univ.ai/univ_data/Training Data.csv')
#test=pd.read_csv('')

In [4]:
def preprocessing(df):
  df=pd.get_dummies(data=df,columns=["house_ownership","married","car_ownership"],drop_first=True)
  df.drop(["profession","city","state"],axis=1,inplace=True)
  return df

In [5]:
train=preprocessing(train)

X = train.drop('risk_flag', axis=1)
y = train['risk_flag']

from imblearn.under_sampling import NeighbourhoodCleaningRule
undersample = NeighbourhoodCleaningRule(n_neighbors=3, threshold_cleaning=0.5)
# transform the dataset
X_under, y_under = undersample.fit_resample(X, y)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/loca

In [6]:
print(X_under.shape,y_under.shape)

(188433, 10) (188433,)


In [14]:
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X_under, y_under, test_size=0.1,shuffle=True)
X_train_new=fu.fit_transform(X_under)
#X_test_new=fu.transform(X_test)
# from sklearn.preprocessing import MinMaxScaler
# scaler=MinMaxScaler()
# X_train=scaler.fit_transform(X_train)
# X_test=scaler.transform(X_test)

In [10]:
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score
gbc=GradientBoostingClassifier()
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(gbc, X_train_new, y_under, scoring='roc_auc', cv=cv, n_jobs=-1)
print("Roc Auc:",np.mean(scores))

Roc Auc: 0.7700071564572486


In [15]:
gbc.fit(X_train_new,y_under)
# y_pred_test=gbc.predict(X_test)
# auc_test = roc_auc_score(y_test, y_pred_test)
# print('ROC AUC test: %f' % auc_test)

ROC AUC test: 0.501935
